In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
customers_df = spark.read.format('csv').option('inferSchema',True).load('/public/trendytech/retail_db/customers/part-00000')

In [4]:
customers_df.show()

+---+-----------+---------+---------+---------+--------------------+-------------+---+-----+
|_c0|        _c1|      _c2|      _c3|      _c4|                 _c5|          _c6|_c7|  _c8|
+---+-----------+---------+---------+---------+--------------------+-------------+---+-----+
|  1|    Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville| TX|78521|
|  2|       Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton| CO|80126|
|  3|        Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas| PR|  725|
|  4|       Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos| CA|92069|
|  5|     Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas| PR|  725|
|  6|       Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic| NJ| 7055|
|  7|    Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|       Caguas| PR|  725|
|  8|      Megan|    Smith|XXXXXXXXX|XXXXXXXXX|3047 Foggy Forest...|  

In [5]:
customers_new_df = customers_df.toDF("customer_id","customer_fname","customer_lastname","customer_email","customer_password","customer_street","customer_city","customer_state","customer_zipcode")

In [6]:
customers_new_df.show()

+-----------+--------------+-----------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lastname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-----------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|        Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|          Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|            Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|          Mary|            Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|    

In [7]:
spark.sql("create database itv017244_bucketingdb")

""


In [10]:
customers_new_df.write.format('parquet').mode('overwrite').bucketBy(4,'customer_id').saveAsTable('itv017244_bucketingdb.customernew')

In [11]:
spark.sql("select count(*) from itv017244_bucketingdb.customernew where customer_state = 'TX'")

count(1)
635


In [12]:
spark.sql("select count(*) from itv017244_bucketingdb.customernew where customer_id = 10")

count(1)
1


In [14]:
customers_new_df.createOrReplaceTempView("customer_nonbucket")

In [15]:
spark.sql("select * from customer_nonbucket where customer_id = 10")

customer_id,customer_fname,customer_lastname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
10,Melissa,Smith,XXXXXXXXX,XXXXXXXXX,8598 Harvest Beac...,Stafford,VA,22554
